In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-prediction-on-movie-reviews/movies.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/sample.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv
/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv


In [82]:
#import necessary libraries
import pandas as pd
import numpy as np
import string
import re

In [83]:
df_train=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/train.csv")
df_train.head()

,movieid,reviewerName,isFrequentReviewer,reviewText,sentiment
0,marvelous_pirate,Benjamin Henry,False,Henry Selick’s first movie since 2009’s Corali...,POSITIVE
1,tony_montana_frodo_baggins_v_rocky_balboa,Felicia Lopez,False,With a cast that reads like the Vogue Oscar pa...,NEGATIVE
2,darth_vader_katniss_everdeen_sorcerer_donnie_d...,Mr. Charles Burgess,True,Creed II does not give us anything but another...,POSITIVE
3,lara_croft_glimmer,Ryan Barrett,False,"I know what you're thinking, but this is no Li...",POSITIVE
4,jason_bourne_surreal_the_terminator_indiana_jones,Alexander Glover,False,Director Fernando Meirelles tells the story wi...,POSITIVE


In [84]:
#drop first 3 columns
df_train=df_train.drop(['movieid','reviewerName','isFrequentReviewer'],axis=1)
df_train.head()

,reviewText,sentiment
0,Henry Selick’s first movie since 2009’s Corali...,POSITIVE
1,With a cast that reads like the Vogue Oscar pa...,NEGATIVE
2,Creed II does not give us anything but another...,POSITIVE
3,"I know what you're thinking, but this is no Li...",POSITIVE
4,Director Fernando Meirelles tells the story wi...,POSITIVE


In [85]:
#fill missing values with blank
df_train=df_train.fillna(" ")
df_train.isnull().sum()

reviewText    0
sentiment     0
dtype: int64

In [86]:
#editing reviewText column

# Lowercasing
df_train['reviewText'] = df_train['reviewText'].str.lower()

# Removing Punctuation
df_train['reviewText'] = df_train['reviewText'].apply(lambda text: re.sub(r'[^\w\s]', '', text))

# Tokenization
df_train['reviewText'] = df_train['reviewText'].apply(lambda text: re.findall(r'\w+', text))

# Removing Stop Words
stop_words = set(['the', 'is', 'and'])  # Add more stop words if needed
df_train['reviewText'] = df_train['reviewText'].apply(lambda tokens: [word for word in tokens if word not in stop_words])


# Removing Numbers
df_train['reviewText'] = df_train['reviewText'].apply(lambda tokens: [word for word in tokens if not word.isdigit()])

# Removing Extra Whitespaces
df_train['reviewText'] = df_train['reviewText'].apply(lambda tokens: [re.sub(r'\s+', ' ', word) for word in tokens])

# Joining Tokens back to Text
df_train['reviewText'] = df_train['reviewText'].apply(' '.join)

# Handling Special Characters or URLs (Example: Removing URLs)
df_train['reviewText'] = df_train['reviewText'].apply(lambda text: re.sub(r'http\S+', '', text))


In [87]:
#selecting X and y 
X=df_train['reviewText']
y=df_train['sentiment']



In [88]:
#import necessary sklearn modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [89]:
#train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)



In [90]:
#vectorize the text using count vectorizer
vectorizer=CountVectorizer()
X_train_vec=vectorizer.fit_transform(X_train)
X_test_vec=vectorizer.transform(X_test)



In [91]:
#Train Logistic Model
model=LogisticRegression()
model.fit(X_train_vec,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [92]:
#make predictions on test set
y_pred=model.predict(X_test_vec)


In [93]:
#calculate accuracy of the model
accuracy=accuracy_score(y_test,y_pred)
print("Accuracy:",accuracy)

Accuracy: 0.7963258785942492


In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

# Split the dataset into dependent(X) and independent variable(y)
X = df_train['reviewText']
y = df_train['sentiment']

# Split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construct a pipeline for text classification
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', LinearSVC())
])

# Set the parameters for the pipeline
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams
    'clf__C': [0.1, 1, 10]
}

# Perform randomized search to find the best parameters
random_search = RandomizedSearchCV(pipeline, parameters, cv=5, n_iter=3)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)


Classification Report:
              precision    recall  f1-score   support

    NEGATIVE       0.78      0.54      0.64     10696
    POSITIVE       0.80      0.93      0.86     21856

    accuracy                           0.80     32552
   macro avg       0.79      0.73      0.75     32552
weighted avg       0.80      0.80      0.79     32552



In [95]:
"""

#Naive Bayes

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

# Split the dataset into dependent (X) and independent variable (y)
X = df_train['reviewText']
y = df_train['sentiment']

# Split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construct a pipeline for text classification
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# Set the parameters for the pipeline
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams
    'clf__alpha': [0.1, 1, 10]  # Try different values for the smoothing parameter alpha
}

# Perform randomized search to find the best parameters
random_search = RandomizedSearchCV(pipeline, parameters, cv=5, n_iter=3)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Evaluate the model
y_pred = best_model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)
"""


"\n\n#Naive Bayes\n\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.naive_bayes import MultinomialNB\nfrom sklearn.metrics import classification_report\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.model_selection import cross_val_score\n\n# Split the dataset into dependent (X) and independent variable (y)\nX = df_train['reviewText']\ny = df_train['sentiment']\n\n# Split the dataset into training and test set\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Construct a pipeline for text classification\npipeline = Pipeline([\n    ('vect', TfidfVectorizer()),\n    ('clf', MultinomialNB())\n])\n\n# Set the parameters for the pipeline\nparameters = {\n    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams\n    'clf__alpha': [0.1, 1, 10]  # Try different values for the smoothing par

In [96]:
"""
#decision tree

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

# Split the dataset into dependent (X) and independent variable (y)
X = df_train['reviewText']
y = df_train['sentiment']

# Split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construct a pipeline for text classification
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

# Set the parameters for the pipeline
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams
    'clf__max_depth': [None, 10, 20]  # Try different values for the maximum depth
}

# Perform randomized search to find the best parameters
random_search = RandomizedSearchCV(pipeline, parameters, cv=5, n_iter=3)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)
print(report)
"""


"\n#decision tree\n\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import classification_report\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.model_selection import RandomizedSearchCV\n\n# Split the dataset into dependent (X) and independent variable (y)\nX = df_train['reviewText']\ny = df_train['sentiment']\n\n# Split the dataset into training and test set\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Construct a pipeline for text classification\npipeline = Pipeline([\n    ('vect', TfidfVectorizer()),\n    ('clf', DecisionTreeClassifier())\n])\n\n# Set the parameters for the pipeline\nparameters = {\n    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams\n    'clf__max_depth': [None, 10, 20]  # Try different values for the maximum depth\n}\n\n# Perform randomized search t

In [97]:
"""
#decision tree

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

# Split the dataset into dependent (X) and independent variable (y)
X = df_train['reviewText']
y = df_train['sentiment']

# Split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construct a pipeline for text classification
pipeline = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

# Set the parameters for the pipeline
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams
    'clf__max_depth': [None, 10, 20]  # Try different values for the maximum depth
}

# Perform randomized search to find the best parameters
random_search = RandomizedSearchCV(pipeline, parameters, cv=5, n_iter=3)
random_search.fit(X_train, y_train)

# Get the best model
best_model = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Generate the classification report
report = classification_report(y_test, y_pred)
print(report)
"""


"\n#decision tree\n\nfrom sklearn.feature_extraction.text import TfidfVectorizer\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import classification_report\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.model_selection import RandomizedSearchCV\n\n# Split the dataset into dependent (X) and independent variable (y)\nX = df_train['reviewText']\ny = df_train['sentiment']\n\n# Split the dataset into training and test set\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n# Construct a pipeline for text classification\npipeline = Pipeline([\n    ('vect', TfidfVectorizer()),\n    ('clf', DecisionTreeClassifier())\n])\n\n# Set the parameters for the pipeline\nparameters = {\n    'vect__ngram_range': [(1, 1), (1, 2)],  # Try both unigrams and bigrams\n    'clf__max_depth': [None, 10, 20]  # Try different values for the maximum depth\n}\n\n# Perform randomized search t

In [98]:
#load the test file

df_test=pd.read_csv("/kaggle/input/sentiment-prediction-on-movie-reviews/test.csv")
df_test.head()




,movieid,reviewerName,isTopCritic,reviewText
0,legend_marty_mcfly_oracle,John Kim,False,Green slowly cranks up the dread with style an...
1,terminator_katniss_everdeen_glimmer,Brian Chaney,False,Philip Noyce's direction is elegant and unforc...
2,james_bond_labyrinth_gollum,Danielle Parker,False,It wouldn't do to say what path Maria ultimate...
3,v_quest_han_solo_wondrous,Brittany Lane,False,Pig is not exactly the arthouse John Wick that...
4,enigma_hulk_surreal_starlight,Justin Willis,False,An imaginative no-budget musical of sorts abou...


In [99]:
#drop first 3 columns
df_test=df_test.drop(['movieid','reviewerName','isTopCritic'],axis=1)
df_test.head()

,reviewText
0,Green slowly cranks up the dread with style an...
1,Philip Noyce's direction is elegant and unforc...
2,It wouldn't do to say what path Maria ultimate...
3,Pig is not exactly the arthouse John Wick that...
4,An imaginative no-budget musical of sorts abou...


In [100]:
#filling null values with blank
df_test['reviewText']=df_test['reviewText'].fillna(" ")

In [101]:
#vectorize the test data using the same Countvectorizer used for training
X_test_vectorized=vectorizer.transform(df_test['reviewText'])


In [102]:
#make prediction on the test data
test_predictions=model.predict(X_test_vectorized)
test_predictions


array(['POSITIVE', 'POSITIVE', 'POSITIVE', ..., 'NEGATIVE', 'POSITIVE',
       'NEGATIVE'], dtype=object)

In [103]:
#making a submission
submission=pd.DataFrame(columns=['id','sentiment'])
submission['id']=[i for i in range(len(test_predictions))]
submission['sentiment']=test_predictions

In [104]:
submission.shape


(55315, 2)

In [105]:
submission.info

<bound method DataFrame.info of           id sentiment
0          0  POSITIVE
1          1  POSITIVE
2          2  POSITIVE
3          3  POSITIVE
4          4  POSITIVE
...      ...       ...
55310  55310  POSITIVE
55311  55311  NEGATIVE
55312  55312  NEGATIVE
55313  55313  POSITIVE
55314  55314  NEGATIVE

[55315 rows x 2 columns]>

In [106]:
submission.head()

,id,sentiment
0,0,POSITIVE
1,1,POSITIVE
2,2,POSITIVE
3,3,POSITIVE
4,4,POSITIVE


In [107]:
#save the submission file
submission.to_csv("submission.csv",index=False)